In [ ]:
import random
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Sequential
import numpy as np
import random

In [ ]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py


--2022-02-03 00:14:45--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2161 (2.1K) [text/plain]
Saving to: ‘roman_numerals.py’

roman_numerals.py   100%[===================>]   2.11K  --.-KB/s    in 0s      

2022-02-03 00:14:46 (35.1 MB/s) - ‘roman_numerals.py’ saved [2161/2161]



In [ ]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py


--2022-02-03 00:14:53--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1812 (1.8K) [text/plain]
Saving to: ‘encoder.py’

encoder.py          100%[===================>]   1.77K  --.-KB/s    in 0s      

2022-02-03 00:14:53 (26.3 MB/s) - ‘encoder.py’ saved [1812/1812]



In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector
import numpy as np

In [ ]:
from encoder import CharacterTable 
import roman_numerals as cnv

# Parameters for the model and dataset.
DATASET_SIZE = 200
INPUT = 14
OUTPUT = 4
# object to encode roman numbers to one-hot 
romans = 'MDCLXVI '
rtable = CharacterTable(romans)

# object to encode arabic numbers to one-hot
chars = '0123456789 '
dtable = CharacterTable(chars)

In [ ]:
seq_samples = []
seq_labels = []
used = []
repetitions = 0
generated = 0
print('Generating data...')
while len(seq_samples) < DATASET_SIZE:
    number = random.randint(1,200)
    # skip if already in the dataset
    if number in used: continue
    used.append(number)
    # roman input
    roman = cnv.convert(number) 
    roman += ' ' * (INPUT - len(roman))
    
    # arabic output
    arabic = number
    
    seq_samples.append(arabic)
    seq_labels.append(roman)
print('Total roman numbers:', len(seq_samples))

Generating data...
Total roman numbers: 200


In [ ]:
# one-hot encoding of all romans and arabic numbers
print('Vectorization...')
labels = np.zeros((DATASET_SIZE, INPUT, len(romans)), dtype=np.bool)

for i, sentence in enumerate(seq_labels):
    labels[i] = rtable.encode(sentence, INPUT)
   
print("labels",labels.shape)

Vectorization...
labels (200, 14, 8)


In [ ]:
samples = np.array(seq_samples)
samples.shape

(200,)

In [ ]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


In [ ]:
OUTPUT_SEQUENCE_LEN=14

model = Sequential()
model.add(Dense(16, input_dim=1) ) 
model.add(RepeatVector(OUTPUT_SEQUENCE_LEN)) #length of the text
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(8,activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy','mae'])
num_epochs = 0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                32        
                                                                 
 repeat_vector (RepeatVector  (None, 14, 16)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 14, 128)           74240     
                                                                 
 lstm_1 (LSTM)               (None, 14, 128)           131584    
                                                                 
 lstm_2 (LSTM)               (None, 14, 128)           131584    
                                                                 
 dense_1 (Dense)             (None, 14, 8)             1032      
                                                        

In [ ]:
def avg(list):
    return sum(list)/len(list)

EPOCHS = 50
# Train the model each generation and show predictions against the validation dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration*EPOCHS)
    H = model.fit(trainSamples, trainLabels, batch_size=128, epochs=EPOCHS, 
                  validation_data=(testSamples, testLabels), verbose=0)
    print("Validation accuracy: {}".format(avg(H.history["val_accuracy"])))
    # Select 10 samples from the validation set at random to visualize errors.
    accuracy = 0

    for i in range(len(testSamples)):
        
           
        pred = model.predict(testSamples)
        
        roman = rtable.decode(testLabels[i])
        correct = roman
        
        number = testSamples[i]

        prediction = rtable.decode(pred[i])

        correctTxt = ""
        if correct == prediction:
            correctTxt+='OK!'
            accuracy+=1

        v = random.random()
        if v>0.9:
            print('expression:',number , end=' ')
            print('correct:', roman, end=' ')
            print('predicted:', prediction, end=' ')
            print(correctTxt)
        
    print('accuracy=',accuracy,"/",len(testSamples),"=",accuracy/len(testSamples))



--------------------------------------------------
Iteration 50
Validation accuracy: 0.7314285624027252
expression: 139 correct: CXXXIX         predicted: XXXXI          
expression: 13 correct: XIII           predicted: XXI            
expression: 126 correct: CXXVI          predicted: XXXII          
expression: 23 correct: XXIII          predicted: XXII           
expression: 199 correct: CXCIX          predicted: XXXXII         
expression: 55 correct: LV             predicted: XXXI           
expression: 77 correct: LXXVII         predicted: XXXI           
expression: 158 correct: CLVIII         predicted: XXXXI          
expression: 76 correct: LXXVI          predicted: XXXI           
expression: 102 correct: CII            predicted: XXXII          
expression: 11 correct: XI             predicted: XXI            
expression: 137 correct: CXXXVII        predicted: XXXXI          
expression: 81 correct: LXXXI          predicted: XXXI           
accuracy= 0 / 100 = 0.0

------